In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import expm, eigh
from tqdm import tqdm
%matplotlib widget


# Electric field functions
def E_omega(t):
    return E0_au * np.exp(-((t-t0_au) / tau_au)**2) * np.cos(omega_au * t + phi1)

def E_2omega(t):
    return E02_au * np.exp(-((t-t0_au) / tau_au)**2) * np.cos(2*omega_au * t + phi2)
# Energies and dipole elements
omega_s_eV, omega_p_eV, omega_es_eV, omega_ep_eV, omega_ed_eV = -24.6, -3.4, 17.8, 17.8, 17.8
d_sp, d_pes, d_sep, d_ped = 0.45, 0.11, 0.2, 0.1
omega_s, omega_p = omega_s_eV*0.0367493, omega_p_eV*0.0367493
omega_es, omega_ep, omega_ed = omega_es_eV*0.0367493, omega_ep_eV*0.0367493, omega_ed_eV*0.0367493

# Initial Hamiltonian without the field
# Initial Hamiltonian without the field
H0 = np.array([
    [omega_s, 0, 0, 0, 0],
    [0, omega_p, 0, 0, 0],
    [0, 0, omega_es, 0, 0],
    [0, 0, 0, omega_ep, 0],
    [0, 0, 0, 0, omega_ed]
], dtype=complex)


# Hamiltonian function
def hamiltonian(t):
    E_om = E_omega(t)
    E_2om = E_2omega(t)
    return np.array([
        [0, d_sp * E_om, 0, d_sep * E_2om, 0],
        [d_sp * E_om, 0, d_pes * E_om, 0, d_ped * E_om],
        [0, d_pes * E_om, 0, 0, 0],
        [d_sep * E_2om, 0, 0, 0, 0],
        [0, d_ped * E_om, 0, 0, 0]
    ], dtype=complex)

# Time array(in fs)
start_time = -150 # fs
stop_time = 400 # fs
# Electric Field constants
ϵ0 = 8.854187817e-12 # F/m
c = 3e8 # m/s
E0_au = 0.0005
E02_au = 0.00005
phi1 = 0.0
phi2 = 0.0
tau = 50  # fs
tau_au = tau*41
t0 = 0 # fs
t0_au = t0*41

In [10]:
steps = 100000
t_values_SI = np.linspace(start_time, stop_time, steps)     # Time array in femtoseconds
t_values = t_values_SI*41 # Time array in atomic units
dt = t_values[1] - t_values[0]


# Function to run simulation for a given omega_eV
def run_simulation(omega_eV):
    omega_eV = omega_au/0.0367493
    # Initial state
    psi_0 = np.array([1, 0, 0, 0, 0], dtype=complex)
    psi_t_list = [psi_0]

    U0_half_dt = expm(-1j * H0 * dt / 2)
    
    # Time evolution
    for index in range(len(t_values)-1):
        t = t_values[index]
        Hamil_t = hamiltonian(t)
        eigenvalues, eigenvectors = eigh(Hamil_t)
        
        # Checks for Hermiticity and unitarity
        assert np.allclose(Hamil_t, Hamil_t.T.conj()), "Hamiltonian is not Hermitian"
        assert np.allclose(np.dot(eigenvectors.T.conj(), eigenvectors), np.eye(len(Hamil_t))), "Eigenvectors are not unitary"

        # Split-operator method
        psi_dt = psi_t_list[index]
        psi_0_U0 = np.dot(U0_half_dt, psi_dt)
        psi_0_eigenbasis = np.dot(eigenvectors.T.conj(), psi_0_U0)
        U_t = np.diag(np.exp(-1j * eigenvalues * dt))
        psi_t_eigenbasis = np.dot(U_t, psi_0_eigenbasis)
        psi_t_pre = np.dot(eigenvectors, psi_t_eigenbasis)
        psi_t_U0 = np.dot(U0_half_dt, psi_t_pre)
        psi_t_list.append(psi_t_U0)

    # Calculate probabilities
    psi_t_array = np.array(psi_t_list)
    prob_s = np.abs(np.array(psi_t_array)[:,0])**2
    prob_p = np.abs(np.array(psi_t_array)[:, 1])**2
    prob_es = np.abs(np.array(psi_t_array)[:, 2])**2
    prob_ep = np.abs(np.array(psi_t_array)[:, 3])**2
    prob_ed = np.abs(np.array(psi_t_array)[:, 4])**2

    # Plot results
    title = f"State probabilities over time for {round(omega_eV, 3)}eV with E_omega = {E0_au}au and E_2omega = {E02_au}fs"
    plt.figure(figsize=(10, 6))
    plt.plot(t_values_SI, prob_s, label='Probability in s state')
    plt.plot(t_values_SI, prob_p, label='Probability in p state')
    plt.plot(t_values_SI, prob_es, label='Probability in es state')
    plt.plot(t_values_SI, prob_ep, label='Probability in ep state')
    plt.plot(t_values_SI, prob_ed, label='Probability in ed state')
    plt.title(title)
    plt.xlabel('Time (fs)')
    plt.ylabel('Probability')
    plt.grid(which='both')
    plt.legend()
    plt.savefig(f"omega_{round(omega_eV, 3)}eV.png")
    plt.close()

# Run simulations for different omega_eV values
omega_values = np.arange(21.1, 21.3, 0.005)
omega_au_values = omega_values*0.0367493
for omega_au in omega_au_values:
    print(f"Running simulation for omega = {round(omega_au/0.0367493, 3)} eV")
    run_simulation(omega_au)
    print(f"Completed simulation for omega = {omega_au/0.0367493} eV")

Running simulation for omega = 21.1 eV
Completed simulation for omega = 21.1 eV
Running simulation for omega = 21.105 eV
Completed simulation for omega = 21.105 eV
Running simulation for omega = 21.11 eV
Completed simulation for omega = 21.11 eV
Running simulation for omega = 21.115 eV
Completed simulation for omega = 21.115 eV
Running simulation for omega = 21.12 eV
Completed simulation for omega = 21.119999999999997 eV
Running simulation for omega = 21.125 eV
Completed simulation for omega = 21.124999999999996 eV
Running simulation for omega = 21.13 eV
Completed simulation for omega = 21.129999999999995 eV
Running simulation for omega = 21.135 eV
Completed simulation for omega = 21.134999999999994 eV
Running simulation for omega = 21.14 eV
Completed simulation for omega = 21.139999999999993 eV
Running simulation for omega = 21.145 eV
Completed simulation for omega = 21.144999999999992 eV
Running simulation for omega = 21.15 eV
Completed simulation for omega = 21.14999999999999 eV
Run